In [17]:
from pathlib import Path
from concurrent.futures import ProcessPoolExecutor
import logging
import sys

from ligandparam.recipes import LazyLigand
from ligandparam import __logging_name__ , __version__
from rdkit import Chem
from rdkit.Chem import AllChem, rdFMCS
import MDAnalysis as mda

home_dir = home_dir = Path("..").resolve()
data_dir = home_dir / "data"
preparation_dir = data_dir / "preparation"

In [19]:
name = "17"
target_dir = Path(preparation_dir, name)

binder = Path(target_dir, f"binder_{name}.pdb")
parametrized_binder = Path(target_dir, f"parametrized_binder_{name}.pdb")

docked = Chem.MolFromPDBFile(str(binder), removeHs=False)
minimized = Chem.MolFromPDBFile(str(parametrized_binder), removeHs=False)

In [26]:

mcs = rdFMCS.FindMCS([docked, minimized])
common_smarts = mcs.smartsString
common_mol = Chem.MolFromSmarts(common_smarts)



In [29]:
# Map the atoms to find the common atoms
match1 = minimized.GetSubstructMatch(common_mol)
match2 = docked.GetSubstructMatch(common_mol)

# Align the molecules based on the common scaffold.
AllChem.AlignMol(minimized, docked, atomMap=list(zip(match2, match1)))

# Write the aligned ligand to a pdb file
Chem.MolToPDB   File(minimized, target_dir/f"aligned.pdb")

In [ ]:
Chem.MolFromMol2File("/home/pb777/reaf_check/jacs/cdk2/data/preparation/cdk2/17_0.mol2")

In [ ]:
Chem.MolToMolFile(a, "/home/pb777/reaf_check/jacs/cdk2/data/preparation/binder_17.pdb")

In [2]:
def set_file_logger(logfilename: Path, logname: str = None, filemode: str = 'a') -> logging.Logger:
    if logname is None:
        logname = Path(logfilename).stem
    logger = logging.getLogger(logname)
    logger.setLevel(logging.INFO)
    formatter = logging.Formatter(
        "{asctime} - {levelname} - {version} {message}",
        style="{",
        datefmt="%Y-%m-%d %H:%M:%S",
        defaults={"version": __version__},
    )
    file_handler = logging.FileHandler(filename=logfilename, mode=filemode)
    file_handler.setLevel(logging.INFO)
    file_handler.setFormatter(formatter)
    logger.addHandler(file_handler)

    return logger

def set_stdout_logger(logger_name: str) -> logging.Logger:
    logger = logging.getLogger(logger_name)
    logger.setLevel(logging.INFO)  # Set the root logger level to DEBUG

    formatter = logging.Formatter(
        "{asctime} - {levelname} - {version} {message}",
        style="{",
        datefmt="%Y-%m-%d %H:%M:%S",
        defaults={"version": ""},
    )

    stream_handler = logging.StreamHandler(sys.stdout)
    stream_handler.setLevel(logging.INFO)
    stream_handler.setFormatter(formatter)
    logger.addHandler(stream_handler)

    return logger

gaussian_paths = {
    "gaussian_root": "/home/pb777/GAUSSIAN",
    "gauss_exedir": "/home/pb777/GAUSSIAN/g16/bsd:/home/pb777/GAUSSIAN/g16",
    "gaussian_binary": "/home/pb777/GAUSSIAN/g16/g16",
    "gaussian_scratch": "/home/pb777/GAUSSIAN/g16/scratch",
}

In [4]:
round((9.65 + 10.92 + 12.09 + 11.52) / 4, 2)

11.04

In [5]:
14:28:07

31.09

In [12]:
def worker(node, data_cwd, gaussian_paths):
    cpx_dir = data_cwd / node
    binder_path = Path(cpx_dir, f"binder_{node}.pdb")
    logger = set_file_logger(cpx_dir / f"{node}.log")
    parametrize_ligand = LazyLigand(in_filename= binder_path, cwd=cpx_dir, logger=logger, net_charge=0, nproc=2,
                                    atom_type = "gaff2", mem=1024, **gaussian_paths)
    parametrize_ligand.setup()
    parametrize_ligand.execute(dry_run=False)

In [ ]:
binders = ["17", "1h1q", "1h1r", "1h1s", "1oi9", "1oiu", "1oiy", "20", "21", "22", "26", "28", "29", "30", "31", "32"]
data_cwd = Path("/home/pb777/reaf_check/jacs/cdk2/data")

with ProcessPoolExecutor(max_workers=32) as ex:
    futuros = {}
    for binder in binders:
        futu = ex.submit(worker, binder, data_cwd, gaussian_paths)
        futuros[binder] = futu

    for node, futu in futuros.items():
        print(f"Waiting for {node}")
        futu.result()

Waiting for 17


In [11]:
len(binders)

16